In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pylab as plt
%matplotlib inline
import pandas as pd
import seaborn as sns

In [2]:
!wget https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad-19/master/labs/lab1/lab1_data/nba.csv

/bin/sh: wget: command not found


In [2]:
data = pd.read_csv('nba.csv')

In [3]:
data.head()

,Unnamed: 0,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
0,7,A. Johnson,ATL,30,34,24,28,64,52
1,10,A. Johnson,TOR,63,57,44,38,120,82
2,15,Abdur-Rahim,SAC,114,124,84,90,238,174
3,22,Afflalo,DEN,236,284,189,227,520,416
4,23,Afflalo,DET,75,88,62,67,163,129


Считая распределение количества результативных бросков дома и на выезде для каждого игрока биномиально распределенной случайной величиной, вычислим $pvalue$ в изначальном прдположении независимости процента голов от локации матча. 

In [4]:
from statsmodels.stats.proportion import proportions_ztest

In [23]:
x = data.values[:, 3:7]

props = []

for i in range(np.shape(x)[0]):
    props.append(proportions_ztest(count = [x[i, 2], x[i, 3]], nobs = [x[i, 0], x[i, 1]], value = 0.0, alternative='two-sided')[1])
    
props[:5]

[0.809816523274713,
 0.7089053094753638,
 0.8478862104448388,
 0.9648717761909615,
 0.3064312956523121]

In [16]:
pr = np.asarray(props)
print(np.sum([pr < 0.05]), np.shape(pr)[0], np.sum([pr < 0.05])/np.shape(pr)[0])

65 936 0.06944444444444445


Как мы видим, в лишь в 65 из 936 случаев можно говорить об обнаружении разницы в успехе бросков у игроков в домашних и гостевых матчах при $\alpha = 0.05$.
Это составляет $6$ процентов случаев.

Если подходить к вопросу как к множественной проверке гипотез, то:

In [18]:
from statsmodels.stats.multitest import multipletests

In [28]:
rejections1 = multipletests(pr, alpha = 0.05, method='holm')[0]
rejections2 = multipletests(pr, alpha = 0.05, method='fdr_bh')[0]

In [29]:
rejected1 = np.sum(rejections1)
rejected2 = np.sum(rejections2)

In [30]:
print('Отвергаем ',rejected1, ' гипотез при использовании метода Холма')
print('Отвергаем ',rejected2, ' гипотез при использовании метода Бенджамини-Хохберга')

Отвергаем  0  гипотез при использовании метода Холма
Отвергаем  0  гипотез при использовании метода Бенджамини-Хохберга


Получаем, что отвергнуто 0 основных гипотез. Поэтому можем утверждать, что на уровне контроля значимости не выше 0.05 не выявлено разницы в успехе бросков игроков дома и в гостях. 

P.s: ни о какой связанности выборок для одного игрока речь не идет, хотя бы потому что у нас различное количество броскрв дома и на выезде.  